In [43]:
import os,sys,copy
from  streamm import buildingblock
import pandas as pd

Read in cply files of building blocks

In [2]:
os.chdir('../donors')

In [5]:
BDTTT = buildingblock.Container('BDTTT')

In [6]:
BDTTT.read_cply()

In [7]:
os.chdir('../acceptors')

In [8]:
TTF = buildingblock.Container('TTF')
TTF.read_cply()

In [10]:
os.chdir('../functional_groups')

In [11]:
EthHex = buildingblock.Container('EthHex')
EthHex.read_cply()

In [12]:
Ester = buildingblock.Container('Ester')
Ester.read_cply()

First attach functionalize TTF with an Ester group

In [14]:
os.chdir('../example')

Find the attaching H and sum the charge on to the carbon

In [15]:
# Find the C-H where H has the bbid R at position 1 
Rkey_i,Xkey_i = Ester.find_XR("R",1,0)

In [17]:
print Ester.particles[Rkey_i].properties['symbol'],Ester.particles[Rkey_i].properties['charge'] 
print Ester.particles[Xkey_i].properties['symbol'],Ester.particles[Xkey_i].properties['charge'] 

H 0.06
C 0.38


In [18]:
Ester.particles[Xkey_i].properties['charge'] += Ester.particles[Rkey_i].properties['charge'] 

In [20]:
Ester.particles[Rkey_i].properties['charge']  = 0.0

In [21]:
print Ester.particles[Rkey_i].properties['symbol'],Ester.particles[Rkey_i].properties['charge'] 
print Ester.particles[Xkey_i].properties['symbol'],Ester.particles[Xkey_i].properties['charge'] 

H 0.0
C 0.44


Do the sam for the TTF 

In [15]:
# Find the C-H where H has the bbid R at position 0 
Rkey_i,Xkey_i = TTF.find_XR("R",0,0)

In [22]:
print TTF.particles[Rkey_i].properties['symbol'],TTF.particles[Rkey_i].properties['charge'] 
print TTF.particles[Xkey_i].properties['symbol'],TTF.particles[Xkey_i].properties['charge'] 

H 0.0
C 0.0


In [23]:
TTF.particles[Xkey_i].properties['charge'] += TTF.particles[Rkey_i].properties['charge'] 
TTF.particles[Rkey_i].properties['charge']  = 0.0

In [24]:
print TTF.particles[Rkey_i].properties['symbol'],TTF.particles[Rkey_i].properties['charge'] 
print TTF.particles[Xkey_i].properties['symbol'],TTF.particles[Xkey_i].properties['charge'] 

H 0.0
C 0.0


Preform attachment 

In [25]:
TTF_Ester = buildingblock.attach(TTF,Ester,"R",0,"R",1)

In [27]:
TTF_Ester.write_xyz()

In [28]:
TTF_Ester.write_cply()

Then attach the BDTTT to the TTF to make the DA repeat unit 

In [29]:
# Find the C-H where H has the bbid R at position 0 
Rkey_i,Xkey_i = BDTTT.find_XR("T",0,0)

In [30]:
# Sum the charges on the attaching carbon
BDTTT.particles[Xkey_i].properties['charge'] += BDTTT.particles[Rkey_i].properties['charge'] 
BDTTT.particles[Rkey_i].properties['charge']  = 0.0

In [31]:
# Change fftype to C! 
BDTTT.particles[Xkey_i].properties['fftype'] = 'C!'

In [32]:
# Find the C-H where H has the bbid R at position 1
Rkey_i,Xkey_i = TTF_Ester.find_XR("T",1,0)

In [33]:
# Sum the charges on the attaching carbon
TTF_Ester.particles[Xkey_i].properties['charge'] += TTF_Ester.particles[Rkey_i].properties['charge'] 
TTF_Ester.particles[Rkey_i].properties['charge']  = 0.0

In [34]:
# Change fftype to C! 
TTF_Ester.particles[Xkey_i].properties['fftype'] = 'C!'

In [35]:
BDTTTT_TTF_Ester = buildingblock.attach(BDTTT,TTF_Ester,"T",0,"T",1)

In [36]:
BDTTTT_TTF_Ester.write_xyz()

In [37]:
BDTTTT_TTF_Ester.write_cply()

This is the repeat unit without alkyl groups and the ESP charges of this unit can be calculated with Gaussian

Now sum the charges of the Ethyl Hexyl on the attaching carbon

In [39]:
# Find the C-H where H has the bbid R at position 1
Rkey_i,Xkey_i = EthHex.find_XR("R",0,0)
# Sum the charges on the attaching carbon
EthHex.particles[Xkey_i].properties['charge'] += EthHex.particles[Rkey_i].properties['charge'] 
EthHex.particles[Rkey_i].properties['charge']  = 0.0

In [41]:
# Check the total charge 
t_q = 0.0 
for k,p in EthHex.particles.iteritems():
    t_q += p.properties['charge']
print t_q

0.0


Attach Ethyl hexyl groups to the three "R" attach points of the BDTTTT_TTF_Ester.

The position number is always 0 since the attach point is deleted when an attach is made

In [54]:
bb_i = copy.deepcopy(BDTTTT_TTF_Ester)
bb_j = copy.deepcopy(EthHex)

In [55]:
bb_i = buildingblock.attach(bb_i,bb_j,"R",0,"R",0)

In [58]:
bb_j = copy.deepcopy(EthHex)
bb_i = buildingblock.attach(bb_i,bb_j,"R",0,"R",0)

In [60]:
bb_j = copy.deepcopy(EthHex)
bb_i = buildingblock.attach(bb_i,bb_j,"R",0,"R",0)

In [69]:
PCE10_n1 = copy.deepcopy(bb_i)

In [70]:
PCE10_n1.tag = 'PCE10_n1'

PCE10_n1.write_xyz()
PCE10_n1.write_cply()

In [72]:
del bb_i,bb_j

Now an oligomer of length n can be made from the repeat unit

In [85]:
n= 5

In [87]:
bb_ij = copy.deepcopy(PCE10_n1)

In [88]:
for n_i in range(n):
    bb_i = copy.deepcopy(bb_ij)
    # Find the C-H where H has the bbid R at position 0
    Rkey_i,Xkey_i = bb_i.find_XR("T",0,0)
    # Sum the charges on the attaching carbon
    bb_i.particles[Xkey_i].properties['charge'] += bb_i.particles[Rkey_i].properties['charge'] 
    bb_i.particles[Rkey_i].properties['charge']  = 0.0
    # Change fftype to C! 
    bb_i.particles[Xkey_i].properties['fftype'] = 'C!'

    bb_j = copy.deepcopy(PCE10_n1)
    # Find the C-H where H has the bbid R at position 1
    Rkey_j,Xkey_j = bb_j.find_XR("T",1,0)
    # Sum the charges on the attaching carbon
    bb_j.particles[Xkey_j].properties['charge'] += bb_j.particles[Rkey_j].properties['charge'] 
    bb_j.particles[Rkey_j].properties['charge']  = 0.0
    # Change fftype to C! 
    bb_j.particles[Xkey_j].properties['fftype'] = 'C!'

    bb_ij =  buildingblock.attach(bb_i,bb_j,"T",0,"T",1)

In [90]:
PCE10_nX = copy.deepcopy(bb_ij)

In [91]:
PCE10_nX.tag = 'PCE10_n%d'%(n)
PCE10_nX.write_xyz()
PCE10_nX.write_cply()